# Calibrate GLU Pricer with `SciPy`

## 导入模型

In [1]:
import numpy as np
# import pandas as pd

import torch 

import sys
sys.path.append('../')
sys.path.append('../../')

# 导入模型
from Networks.nn import NN_pricing_GLU
from Pricing.rBergomi.rBergomi_utils import *
from calibrate_utils import *

# 评估使用 cpu
device = torch.device('cpu')

# 设置参数为 4 个
hyperparams = { 
    'input_dim': 4, 
    'hidden_dim': 64, 
    'hidden_nums': 4,
    'output_dim': 88
}

model = NN_pricing_GLU(hyperparams=hyperparams).to(device=device, dtype=torch.float64)


model_state = torch.load( 
    r'../../Results/models/GLU_rBergomi_flat_forward_variance.pth'
)
model.load_state_dict(model_state)

# 设置为 eval mode
model.eval()
model.to(device=device, dtype=torch.float64)

NN_pricing_GLU(
  (layer_lst): ModuleList(
    (0): GatedBlock(
      (projection): Linear(in_features=4, out_features=64, bias=True)
      (gate): Linear(in_features=4, out_features=64, bias=True)
    )
    (1-3): 3 x GatedBlock(
      (projection): Linear(in_features=64, out_features=64, bias=True)
      (gate): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (output_layer): Linear(in_features=64, out_features=88, bias=True)
  (norms): ModuleList(
    (0-3): 4 x LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
)

## 数据

In [3]:
xx, yy, strikes, maturities = data_read(r"../../Data/rBergomiTrainSet.txt.gz")

# 训练集的 Upper and Lower Bounds
upper_bound = np.array([0.16,4,-0.1,0.5])
lower_bound = np.array([0.01,0.3,-0.95,0.025])

网格数据形状：(40000, 92)
参数形状：(40000, 4)
隐含波动率曲面形状：(40000, 88)
参数上界: [ 0.16      4.       -0.100011  0.499998]
参数下界: [ 0.0100133  0.300028  -0.949934   0.0250066]


In [4]:
x_train_transform, y_train_transform, x_test_transform, y_test_transform = get_transform_train_test_data(xx, yy, upper_bound, lower_bound, 0.15)

## `SciPy` 校准

In [6]:
def CostFunc(x,sample_ind):
    return np.sum( 
        np.power((model(torch.from_numpy(x)).detach().numpy() - y_test_transform[sample_ind]),2)
    )

Approx_scipy, Timing_scipy = calibrate_with_scipy(CostFunc, x_test_transform, upper_bound, lower_bound)

SciPy 优化结果 (前 3 轮):
 [[[ 0.10466409  0.59075522 -0.73500467  0.24551838]
  [ 0.10466411  0.59075495 -0.73500607  0.24551919]
  [ 0.10466409  0.59075519 -0.73500471  0.24551839]]

 [[ 0.15582185  3.37345513 -0.46182846  0.29554069]
  [ 0.15582191  3.37345606 -0.46182845  0.29554073]
  [ 0.15582185  3.37345512 -0.46182846  0.29554068]]

 [[ 0.09244891  1.97462469 -0.64085518  0.14207235]
  [ 0.09244891  1.9746248  -0.64085522  0.1420723 ]
  [ 0.09244891  1.9746247  -0.64085518  0.14207234]]]
SciPy 优化时间 (前 3 轮):
 [array([0.06140971, 0.04436302, 0.18264985]), array([0.04080391, 0.03954601, 0.13630366]), array([0.0421412 , 0.03490448, 0.11012197])]


In [ ]:
methods=["L-BFGS-B ","SLSQP","BFGS"]

Approx_scipy_lst=[Approx_scipy[i][0] for i in range(len(Approx_scipy))]
Timing_scipy_lst=[Timing_scipy[i][0] for i in range(len(Timing_scipy))]

for i in range(len(Approx_scipy)):
    np.loadtxt(f"../../Results/Calibration/SciPy/rBergomi_GLU_calibrate_scipy_quasi_newton_{methods[i]}.txt", Approx_scipy_lst[i])
    np.savetxt(f"../../Results/Calibration/SciPy/rBergomi_GLU_calibrate_scipy_quasi_newton_{methods[i]}_time.txt", Timing_scipy_lst[i])


ValueError: Expected 1D or 2D array, got 3D array instead